In [2]:
import json
from collections import defaultdict


In [3]:
with open('./alphafold.json', 'r') as file:
    data = json.load(file)

In [4]:
fan_in = defaultdict(int)
fan_out = defaultdict(int)

for module_name, module_info in data.items():
    if "imported_by" in module_info:
        fan_in[module_name] = len(module_info["imported_by"])
    else:
        fan_in[module_name] = 0

    if "imports" in module_info:
        fan_out[module_name] = len(module_info["imports"])
    else:
        fan_out[module_name] = 0

print(f"{'Module':<30} {'Fan-In':<10} {'Fan-Out':<10}")
print("-" * 50)
for module_name in sorted(data.keys()):
    print(f"{module_name:<30} {fan_in[module_name]:<10} {fan_out[module_name]:<10}")

Module                         Fan-In     Fan-Out   
--------------------------------------------------
alphafold                      5          0         
alphafold.common               5          4         
alphafold.data                 2          5         
alphafold.model                3          4         
alphafold.relax                2          5         
alphafold.version              1          0         
numpy                          21         19        
numpy.__config__               1          2         
numpy._distributor_init        1          1         
numpy._globals                 2          0         
numpy._pytesttester            10         2         
numpy.compat                   5          2         
numpy.core                     11         8         
numpy.ctypeslib                1          2         
numpy.dtypes                   2          2         
numpy.exceptions               2          0         
numpy.fft                      2          4     

In [5]:
print("\n=== Highly Coupled Modules and Their Dependencies ===")

# Define the threshold for identifying highly coupled modules
threshold = 10
highly_coupled = [
    (module, fan_in[module] + fan_out[module], fan_in[module], fan_out[module])
    for module in data
    if fan_in[module] + fan_out[module] > threshold
]

# Sort modules by total coupling in descending order
highly_coupled.sort(key=lambda item: item[1], reverse=True)

# Display results
for module, total_coupling, fin, fout in highly_coupled:
    print(f"\nModule: {module}")
    print(f"Total Coupling (Fan-In + Fan-Out): {total_coupling} (Fan-In: {fin}, Fan-Out: {fout})")
    print(f"Imported by: {data[module].get('imported_by', [])}")
    print(f"Imports: {data[module].get('imports', [])}")


=== Highly Coupled Modules and Their Dependencies ===

Module: numpy
Total Coupling (Fan-In + Fan-Out): 40 (Fan-In: 21, Fan-Out: 19)
Imported by: ['alphafold.common', 'alphafold.data', 'alphafold.model', 'alphafold.relax', 'numpy', 'numpy.__config__', 'numpy._distributor_init', 'numpy._pytesttester', 'numpy.compat', 'numpy.core', 'numpy.ctypeslib', 'numpy.dtypes', 'numpy.fft', 'numpy.lib', 'numpy.linalg', 'numpy.ma', 'numpy.matrixlib', 'numpy.polynomial', 'numpy.random', 'numpy.testing', 'run_alphafold.py']
Imports: ['numpy', 'numpy.__config__', 'numpy._distributor_init', 'numpy._globals', 'numpy._pytesttester', 'numpy.compat', 'numpy.core', 'numpy.ctypeslib', 'numpy.dtypes', 'numpy.exceptions', 'numpy.fft', 'numpy.lib', 'numpy.linalg', 'numpy.ma', 'numpy.matrixlib', 'numpy.polynomial', 'numpy.random', 'numpy.testing', 'numpy.version']

Module: numpy.core
Total Coupling (Fan-In + Fan-Out): 19 (Fan-In: 11, Fan-Out: 8)
Imported by: ['numpy', 'numpy.__config__', 'numpy.core', 'numpy.ctyp

In [6]:
print("\n=== Cyclic Dependencies ===")

# Construct adjacency list for the graph
dependency_graph = defaultdict(list)
for module, details in data.items():
    dependency_graph[module].extend(details.get("imports", []))

# Function to detect cycles using DFS
def detect_cycles(graph):
    visited, rec_stack = set(), set()
    detected_cycles = []

    def explore(node, path):
        visited.add(node)
        rec_stack.add(node)
        path.append(node)

        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                explore(neighbor, path.copy())
            elif neighbor in rec_stack:
                cycle_index = path.index(neighbor)
                detected_cycles.append(path[cycle_index:] + [neighbor])  # Form the cycle

        rec_stack.remove(node)

    # Traverse all modules to ensure complete exploration
    for mod in data.keys():
        if mod not in visited:
            explore(mod, [])

    return detected_cycles

# Run cycle detection and display results
cycles_found = detect_cycles(dependency_graph)
if cycles_found:
    print("Cyclic Dependencies Detected:")
    for cycle in cycles_found:
        print(" -> ".join(cycle))
else:
    print("No Cyclic Dependencies Detected.")


=== Cyclic Dependencies ===
Cyclic Dependencies Detected:
alphafold.common -> alphafold.common
numpy -> numpy
numpy -> numpy.__config__ -> numpy
numpy -> numpy.__config__ -> numpy.core -> numpy
numpy -> numpy.__config__ -> numpy.core -> numpy._pytesttester -> numpy
numpy -> numpy.__config__ -> numpy.core -> numpy._pytesttester -> numpy.testing -> numpy
numpy._pytesttester -> numpy.testing -> numpy._pytesttester
numpy.core -> numpy._pytesttester -> numpy.testing -> numpy.core
numpy -> numpy.__config__ -> numpy.core -> numpy._pytesttester -> numpy.testing -> numpy.lib -> numpy
numpy._pytesttester -> numpy.testing -> numpy.lib -> numpy._pytesttester
numpy -> numpy.__config__ -> numpy.core -> numpy._pytesttester -> numpy.testing -> numpy.lib -> numpy.compat -> numpy
numpy.compat -> numpy.compat
numpy.core -> numpy._pytesttester -> numpy.testing -> numpy.lib -> numpy.core
numpy.lib -> numpy.lib
numpy -> numpy.__config__ -> numpy.core -> numpy._pytesttester -> numpy.testing -> numpy.lib -> 

In [8]:
print("\n=== Depth of Dependencies ===")

# Function to determine the longest dependency path using DFS
def find_max_depth(dependency_graph, root):
    explored = set()
    depth_tracker = 0

    def traverse(node, current_depth):
        nonlocal depth_tracker
        explored.add(node)
        depth_tracker = max(depth_tracker, current_depth)

        for adjacent in dependency_graph.get(node, []):
            if adjacent not in explored:
                traverse(adjacent, current_depth + 1)

    traverse(root, 0)
    return depth_tracker

# Determine depth from the primary entry file
entry_file = "run_alphafold.py"
if entry_file in dependency_graph:
    dependency_depth = find_max_depth(dependency_graph, entry_file)
    print(f"Maximum Dependency Depth from {entry_file}: {dependency_depth}")
else:
    print(f"Entry file {entry_file} is not present in the dependency graph.")

# Find the overall maximum dependency depth
overall_max_depth = max(find_max_depth(dependency_graph, module) for module in dependency_graph)
print(f"Maximum Dependency Depth in the Entire Graph: {overall_max_depth}")


=== Depth of Dependencies ===
Maximum Dependency Depth from run_alphafold.py: 8
Maximum Dependency Depth in the Entire Graph: 8
